In [1]:
%matplotlib inline
# Dependencies
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key
from datetime import datetime
import time
from scipy.stats import linregress

In [2]:
#Generate a new set of 1,500 random latitudes and longitudes.

lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 

In [3]:
#print coordinates from the zip lat/long file
coordinates = list(lat_lngs)
print(coordinates[:20])

[(27.010953989861562, 87.29970355146719), (52.39129440890807, 179.10804701963917), (1.3928543673743548, 75.46990580262064), (0.47526252335531183, 54.539480563023886), (-62.291364093913906, 31.00868773401635), (46.51374868880774, 40.41505345476784), (56.29893560224477, -145.98047008729367), (67.37975964923422, -134.2964137207843), (83.06629299143708, -174.60194751741182), (44.78828423004546, -4.870882304728099), (56.32129132393703, 63.0971149679485), (65.28410012703574, -155.18964647737732), (64.94442993660823, -151.92933746591342), (-45.917293096830136, -125.90867275864055), (42.38299108268001, 38.41758150178117), (56.60654161928363, 76.9428311302525), (21.848481386317033, 91.62480843480114), (70.96472935950149, -92.46378812028566), (-13.583179041909858, 164.18563491063225), (11.456899185572226, 78.12274120985018)]


In [4]:
# Get the nearest city using the citipy module
cities = list()

for x in coordinates:
    cities_name = citipy.nearest_city(x[0], x[1]).city_name
    if cities_name not in cities:
        cities.append(cities_name)
print(cities[:10],len(cities))

['dhankuta', 'beringovskiy', 'hithadhoo', 'victoria', 'port elizabeth', 'mechetinskaya', 'palmer', 'aklavik', 'mys shmidta', 'llanes'] 621


In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688


In [17]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval-----")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1


# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city
    # city_weather = requests.get(city_url)
    print(city_url)
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
 # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_description = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
#if it is raining, get the amount of rainfall in inches for the last three hours. If it is not raining, add 0 inches for the city   
        try:
            #city_weather['weather'][0]['main'] == 'Rain'
            city_rain = city_weather['rain']['3h']
        
        except:
            city_rain = 0
            
# if it is snowing, get the amount of snow in inches for the last three hours. If it is not snowing, add 0 inches for the city.
        try:
            #city_weather['weather'][0]['main'] == 'Snow'
            city_snow = city_weather['snow']['3h']
        
        except:
            city_snow = 0

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Date": city_date,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Rain inches(last 3hrs)": city_rain,
                          "Snow inches(last 3hrs)": city_snow,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval-----
-----------------------------
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=dhankuta
Processing Record 1 of Set 1 | dhankuta
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=beringovskiy
Processing Record 2 of Set 1 | beringovskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=hithadhoo
Processing Record 3 of Set 1 | hithadhoo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=victoria
Processing Record 4 of Set 1 | victoria
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=port elizabeth
Processing Record 5 of Set 1 | port elizabeth
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=mechetinskaya
Processing Record 6 of Set 1 | mechetinsk

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=rongcheng
Processing Record 5 of Set 2 | rongcheng
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=itarema
Processing Record 6 of Set 2 | itarema
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=ponta delgada
Processing Record 7 of Set 2 | ponta delgada
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=coihaique
Processing Record 8 of Set 2 | coihaique
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=provideniya
Processing Record 9 of Set 2 | provideniya
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=atambua
Processing Record 10 of Set 2 | atambua
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=butaritari
Processing Record 10 of Set 3 | butaritari
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=pangnirtung
Processing Record 11 of Set 3 | pangnirtung
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=avera
Processing Record 12 of Set 3 | avera
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=la ronge
Processing Record 13 of Set 3 | la ronge
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=laval
Processing Record 14 of Set 3 | laval
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=ribeira grande
Processing Record 15 of Set 3 | ribeira grande
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=kamenka
Processing Record 14 of Set 4 | kamenka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=nara
Processing Record 15 of Set 4 | nara
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=ngukurr
Processing Record 16 of Set 4 | ngukurr
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=bazarnyye mataki
Processing Record 17 of Set 4 | bazarnyye mataki
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=labutta
Processing Record 18 of Set 4 | labutta
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=parabel
Processing Record 19 of Set 4 | parabel
http://api.openweathermap.

Processing Record 18 of Set 5 | palabuhanratu
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=envira
Processing Record 19 of Set 5 | envira
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=hilo
Processing Record 20 of Set 5 | hilo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=chingirlau
Processing Record 21 of Set 5 | chingirlau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=jiaojiang
Processing Record 22 of Set 5 | jiaojiang
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=belushya guba
Processing Record 23 of Set 5 | belushya guba
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f

...

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=ust-kut
Processing Record 33 of Set 8 | ust-kut
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=meyungs
Processing Record 34 of Set 8 | meyungs
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=jinchengjiang
Processing Record 35 of Set 8 | jinchengjiang
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=baturaja
Processing Record 36 of Set 8 | baturaja
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=marovoay
Processing Record 37 of Set 8 | marovoay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=nantucket
Processing Record 38 of Set 8 | nantucket
http://api.openwea

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=maarianhamina
Processing Record 36 of Set 9 | maarianhamina
City not found. Skipping...
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=saint anthony
Processing Record 37 of Set 9 | saint anthony
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=adrar
Processing Record 38 of Set 9 | adrar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=honiara
Processing Record 39 of Set 9 | honiara
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=isiro
Processing Record 40 of Set 9 | isiro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=benghazi
Processing Record 41 of Set 9 | benghazi
http://api.openweathermap.org/data/2.5/weather?u

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=rafael lucio
Processing Record 40 of Set 10 | rafael lucio
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=daru
Processing Record 41 of Set 10 | daru
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=medyn
Processing Record 42 of Set 10 | medyn
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=mackay
Processing Record 43 of Set 10 | mackay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=bhadra
Processing Record 44 of Set 10 | bhadra
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=tiznit
Processing Record 45 of Set 10 | tiznit
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=iranshahr
Processing Record 44 of Set 11 | iranshahr
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=prescott
Processing Record 45 of Set 11 | prescott
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=yamada
Processing Record 46 of Set 11 | yamada
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=sur
Processing Record 47 of Set 11 | sur
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=wukari
Processing Record 48 of Set 11 | wukari
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=montoro
Processing Record 49 of Set 11 | montoro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=inhambane
Processing Record 49 of Set 12 | inhambane
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=makhinjauri
Processing Record 50 of Set 12 | makhinjauri
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=alta floresta
Processing Record 1 of Set 13 | alta floresta
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=puerto escondido
Processing Record 2 of Set 13 | puerto escondido
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=yar-sale
Processing Record 3 of Set 13 | yar-sale
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=f8a8f3743cd9de275d97a88345ed6688&q=paragominas
Processing Record 4 of Set 13 | paragominas
http://api.openweathermap.org/data/2.5/w

In [18]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
0,Dhankuta,NP,2020-01-05 20:52:22,26.98,87.34,38.93,82,12,5.14,0.00,0.0,few clouds
1,Beringovskiy,RU,2020-01-05 20:55:55,63.05,179.32,-7.46,79,89,16.67,0.00,0.0,overcast clouds
2,Hithadhoo,MV,2020-01-05 20:55:55,-0.60,73.08,81.48,78,100,11.25,0.00,0.0,overcast clouds
3,Victoria,BN,2020-01-05 20:55:55,5.28,115.24,80.60,74,75,6.93,0.00,0.0,broken clouds
4,Port Elizabeth,US,2020-01-05 20:54:02,39.31,-74.98,43.00,48,90,10.29,0.00,0.0,overcast clouds
5,Mechetinskaya,RU,2020-01-05 20:52:24,46.77,40.45,38.19,63,100,4.45,0.19,0.0,light rain
6,Palmer,AU,2020-01-05 20:54:03,-34.85,139.16,61.00,62,1,6.93,0.00,0.0,clear sky
7,Aklavik,CA,2020-01-05 20:55:55,68.22,-135.01,-16.60,62,75,10.29,0.00,0.0,broken clouds
8,Llanes,ES,2020-01-05 20:52:24,43.42,-4.75,46.99,10,0,11.77,0.00,0.0,clear sky
9,Dalmatovo,RU,2020-01-05 20:52:24,56.26,62.94,18.39,93,0,9.78,0.00,0.0,clear sky


In [19]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description", "Rain inches(last 3hrs)", "Snow inches(last 3hrs)"]
city_data_df = city_data_df[new_column_order]

In [9]:
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches(last 3hrs),Snow inches(last 3hrs)
0,Dhankuta,NP,2020-01-05 20:52:22,26.98,87.34,38.93,82,12,5.14,few clouds,0,0
1,Beringovskiy,RU,2020-01-05 20:44:10,63.05,179.32,-7.46,79,89,16.67,overcast clouds,0,0
2,Hithadhoo,MV,2020-01-05 20:41:28,-0.60,73.08,81.48,78,100,11.25,overcast clouds,0,0
3,Victoria,BN,2020-01-05 20:44:05,5.28,115.24,80.60,74,75,6.93,broken clouds,0,0
4,Port Elizabeth,US,2020-01-05 20:42:18,39.31,-74.98,43.00,41,90,23.04,overcast clouds,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
551,Igdir,IR,2020-01-05 20:53:23,37.52,45.24,21.20,100,20,4.70,mist,0,0
552,Dajal,PK,2020-01-05 20:53:23,29.56,70.38,51.51,87,100,18.01,light rain,0,0
553,Abai,KZ,2020-01-05 20:53:23,41.35,68.95,37.40,86,100,4.70,mist,0,0
554,Baykit,RU,2020-01-05 20:44:24,61.68,96.39,9.34,96,100,4.05,overcast clouds,0,0


In [20]:
# Create the output file (CSV).
output_data_file = "Data/WeatherPy_challenge1.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches(last 3hrs),Snow inches(last 3hrs)
0,Dhankuta,NP,2020-01-05 20:52:22,26.98,87.34,38.93,82,12,5.14,few clouds,0.0,0.0
1,Beringovskiy,RU,2020-01-05 20:55:55,63.05,179.32,-7.46,79,89,16.67,overcast clouds,0.0,0.0
2,Hithadhoo,MV,2020-01-05 20:55:55,-0.60,73.08,81.48,78,100,11.25,overcast clouds,0.0,0.0
3,Victoria,BN,2020-01-05 20:55:55,5.28,115.24,80.60,74,75,6.93,broken clouds,0.0,0.0
4,Port Elizabeth,US,2020-01-05 20:54:02,39.31,-74.98,43.00,48,90,10.29,overcast clouds,0.0,0.0


In [22]:
type(city_data_df)

pandas.core.frame.DataFrame

In [23]:
# How many cities have recorded rainfall?
city_rain = len(city_data_df[city_data_df["Rain inches(last 3hrs)"]>0])
city_rain


66

In [24]:
#How many cities have recorded snow
city_snow = len(city_data_df[city_data_df["Snow inches(last 3hrs)"]>0])
city_snow

14